In [1]:
# simple genai app using langchain
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# langchain tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [3]:
# data ingestion - from the website we need to scrap the data

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
docs = loader.load()

# data splitting

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [5]:
# converting documents to vectors
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore_db = FAISS.from_documents(documents, embeddings)

In [10]:
# Query from a vector store db
query = "What are the two usage limits that langsmith has ? Explain in detail"
result = vectorstore_db.similarity_search(query)
result[0].page_content

'your use case with LangSmith. For example, if you run evals as part of CI/CD in dev or staging, you may\nwant to be more liberal with your usage limits to avoid test failures.\nNow that our limits are set, we can see that LangSmith shows a maximum spend estimate across all workspaces:'

In [7]:
# retrieval chain, document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_template(
    """
        Answer the following question based only on the provided context:
        <context> 
        {context}
        </context>
    """
)


document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n        <context> \n        {context}\n        </context>\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x15900f550>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1590d2290>, root_client=<openai.OpenAI object at 0x137ab02e0>, root_async_client=<openai.AsyncOpenAI object at 0x137ab0f70>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stu

In [11]:
# from langchain_core.documents import Document
# query_result = document_chain.invoke({
#     "input":query,
#     "context":[Document(page_content="LangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we've been tracking on our usage graph.")]
# })

In [12]:
query_result

'LangSmith has usage limits related to two metrics: total traces and extended retention traces.'

In [13]:
# retreiver

retriever = vectorstore_db.as_retriever()

from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [14]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13e96ff40>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n        <context> \n        {context}\n        </context>\n    '), additional_k

In [15]:
# get the response from the llm
response = retriever_chain.invoke({"input":query})

In [17]:
response["answer"]

'What do LangSmith Traces (Base Charge) and LangSmith Traces (Extended Data Retention Upgrades) refer to in the context of LangSmith usage and billing? \n\nLangSmith Traces (Base Charge) and LangSmith Traces (Extended Data Retention Upgrades) are the two usage metrics that LangSmith charges for. The Base Charge likely refers to the cost associated with logging traces, which are records of API requests made by applications. The Extended Data Retention Upgrades are likely additional charges incurred when opting for longer retention of these traces beyond the standard period.'

In [18]:
response

{'input': 'What are the two usage limits that langsmith has ? Explain in detail',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='your use case with LangSmith. For example, if you run evals as part of CI/CD in dev or staging, you may\nwant to be more liberal with your usage limits to avoid test failures.\nNow that our limits are set, we can see that LangSmith shows a maximum spend estimate across all workspaces:'),
  Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Understand your current 